# Importação das bibliotecas

Instalação de bibliotecas utilizadas no pré processamento

In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorflow

In [7]:
import pandas as pd
import numpy as np
import nltk
import string  
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('portuguese'))
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Abrindo o dataframe 

In [8]:
df = pd.read_excel('../base/base.xlsx', index_col=0)

# Limpeza dos dados

Nesta etapa o foco é limpar da base de dados os dados que não serão utilizados.
Portanto, com essa função são removidos todos os comentários que tenham autoria do BTG Pactual e fotos de terceiros que marcavam a companhia.

In [9]:
# Esta função remove os dados que não serão utilizados para a análise
def clean_data(df):
    # Remove as aspas dos nomes das colunas
    df = df.rename(columns=lambda x: x.replace('\"', ''))
    # Remove comentários feito pelo btg
    dfWithoutAutor = df[df["autor"] != "btgpactual"]
    # Remove comentários nulos
    dfWithoutNull = dfWithoutAutor[dfWithoutAutor["autor"].notnull()]
    # Deixando apenas as interações do tipo comentário e resposta
    dfFinal = dfWithoutNull.loc[(dfWithoutNull['tipoInteracao'] == 'comentário') | (dfWithoutNull['tipoInteracao'] == 'resposta')]
    return dfFinal

#### Caso de teste

In [10]:
# Criando um data frame de teste para validar o pré processamento
data_test = {'id': [1, 2, 3, 4, 5],
             'dataPublicada': ['03/01/2022', '03/04/2022', '03/07/2022', '03/10/2022', '31/12/2022'],
             'autor': ['btgpactual', 'robert87', 'grace46', 'william25', 'john23'],
             'texto': ["Obrigado, 🚀🚀💙", "Bom dia, gostaria de saber como faço pra conseguir falar com um atendente humanizado no chat do btg banking?", 
                       "💙Bom dia, preciso urgente do meu informe de rendimentos do BTLG11 , para declaração do meu IR. Já enviei vários e-mails e até o momento sem resposta. Telefone ninguém atende. Como faço? 🐸", "Pessoal, sempre muito bom mas se eu puder solicitar uma coisa seria legendas 😅", "Sempre é aproveitoso parar o q tá fazendo para ouvi-lo"],
             'sentimento': ['NEUTRAL', 'NEUTRAL', 'NEGATIVE', 'POSITIVE', 'POSITIVE'],
             'tipoInteracao': ["comentário", "comentário", "comentário", "marcação", "comentário"],
             'anomalia': [1, 0, 0, 1, 1],
             'probabilidadeAnomalia': [100, 20, 30, 100, 100],
             'linkPost': ['https://www.instagram.com/p/CapXhB5Lvas/#17935401550894179', 'https://www.instagram.com/p/CaqBiRpsYLm/', 'https://www.instagram.com/p/CaqBiRpsYLm/', 'https://www.instagram.com/p/CapXhB5Lvas/#17926096268158628', 'https://www.instagram.com/p/CaR1j2TuPpD/#17914683365221503'],
             'processado': [0,0,0,1,1],
             'contemHyperlink': [0,0,0,1,1]}
               
df_testing = pd.DataFrame(data_test)

In [11]:
# Testando a função
clean_data_test = df_testing
cleaning_test = clean_data(clean_data_test)

Os resultados foram colocados em células separadas para facilitar a visualização.

In [12]:
# Entrada do caso de teste
clean_data_test

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink
0,1,03/01/2022,btgpactual,"Obrigado, 🚀🚀💙",NEUTRAL,comentário,1,100,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0
1,2,03/04/2022,robert87,"Bom dia, gostaria de saber como faço pra conse...",NEUTRAL,comentário,0,20,https://www.instagram.com/p/CaqBiRpsYLm/,0,0
2,3,03/07/2022,grace46,"💙Bom dia, preciso urgente do meu informe de re...",NEGATIVE,comentário,0,30,https://www.instagram.com/p/CaqBiRpsYLm/,0,0
3,4,03/10/2022,william25,"Pessoal, sempre muito bom mas se eu puder soli...",POSITIVE,marcação,1,100,https://www.instagram.com/p/CapXhB5Lvas/#17926...,1,1
4,5,31/12/2022,john23,Sempre é aproveitoso parar o q tá fazendo para...,POSITIVE,comentário,1,100,https://www.instagram.com/p/CaR1j2TuPpD/#17914...,1,1


In [13]:
# Resultado do caso de teste
cleaning_test

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink
1,2,03/04/2022,robert87,"Bom dia, gostaria de saber como faço pra conse...",NEUTRAL,comentário,0,20,https://www.instagram.com/p/CaqBiRpsYLm/,0,0
2,3,03/07/2022,grace46,"💙Bom dia, preciso urgente do meu informe de re...",NEGATIVE,comentário,0,30,https://www.instagram.com/p/CaqBiRpsYLm/,0,0
4,5,31/12/2022,john23,Sempre é aproveitoso parar o q tá fazendo para...,POSITIVE,comentário,1,100,https://www.instagram.com/p/CaR1j2TuPpD/#17914...,1,1


#### Aplicação da função no dataframe


Nesta etapa será demonstrado o funcionamento da função na base de dados real 

In [14]:
cleaning_output = clean_data(df)
cleaning_output

,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink
id,,,,,,,,,,
4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0
5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0
6,"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,POSITIVE,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0
11,"""2022-03-03 18:56:00""",claudiofalavinha,@thaotinhasbfc,NEUTRAL,comentário,0,41,https://www.instagram.com/p/CapXhB5Lvas/#17942...,0,0
18,"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,NEUTRAL,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0
...,...,...,...,...,...,...,...,...,...,...
3042413,"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,POSITIVE,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0
3042713,"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,NEGATIVE,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0
3043812,"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,POSITIVE,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0


Nesta etapa as frases são tokenizadas (cada palavra da frase é dividida, sendo 1 palavra = 1 token) e removidas as stop words, palavras que não acrescentam significado semântico relevante (e.g: de, para, ou). Além disso, é gerado uma nova coluna que contém os tratamento citados acima aplicado aos comentários (coluna 'texto'). 

In [15]:
stop_words = list(stopwords.words('portuguese'))

In [16]:
# Defina as stop words personalizadas aqui
custom_stop_words = ['btg', 'pactual', 'btgpactual','pq', 'q', 'pra', 'vcs', 'vc', 'i', 'p', 'kkk', 'y', 'of', 'n'] # Adicione as palavras que deseja remover

In [17]:
stop_words = stop_words + custom_stop_words

In [18]:
def processing_comments(frase):
    #divide as frases em palavrasa
    palavras = frase.split() 
 
    #coloca todas as letras em minúsculas (lower) e remove as stop words
    filtered_words = [word.lower() for word in palavras if word.lower() not in stop_words
                        and not word.startswith('@') and word.isalpha()] 
    #tokeniza as palavras e organiza cada comentário em uma lista de palavras
    tokens = word_tokenize(" ".join(filtered_words)) 
    return tokens

#### Caso de teste

In [19]:
# Testando a função 
processing_test = cleaning_test['texto']
tokenize_test = processing_test.apply(processing_comments)

In [20]:
# Input da função
processing_test

1    Bom dia, gostaria de saber como faço pra conse...
2    💙Bom dia, preciso urgente do meu informe de re...
4    Sempre é aproveitoso parar o q tá fazendo para...
Name: texto, dtype: object

In [21]:
# Resultado da função
tokenize_test

1    [bom, gostaria, saber, faço, conseguir, falar,...
2    [preciso, urgente, informe, rendimentos, decla...
4            [sempre, aproveitoso, parar, tá, fazendo]
Name: texto, dtype: object

####Aplicação da função
Nesta etapa será demonstrado o funcionamento da função na base de dados real

In [ ]:
processing_output = cleaning_output['texto'].apply(processing_comments)
processing_output

In [ ]:
processing_output.to_csv('..\\output\\pre_processamento.csv',encoding='utf-8', index=False, header=True, )